In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**LOAD DATA**

In [ ]:
train_data = pd.read_csv("/kaggle/input/challenges-in-representation-learning-facial-expression-recognition-challenge/train.csv")
train_data.head(5)

In [ ]:
train_data["emotion"].value_counts()

> 0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral)

In [ ]:
test_data = pd.read_csv("/kaggle/input/challenges-in-representation-learning-facial-expression-recognition-challenge/test.csv")
test_data.head(5)

In [ ]:
facialexpression_df = pd.read_csv('/kaggle/input/challenges-in-representation-learning-facial-expression-recognition-challenge/icml_face_data.csv')

In [ ]:
facialexpression_df

In [ ]:
facialexpression_df[' pixels'][0]

> HERE WE SEE THAT PIXELS ARE IN STRING FORMET WE HAVE TO CONVERT THEM INTO ARRAY FORMET

In [ ]:
def string2array(x):
  return np.array(x.split(' ')).reshape(48, 48, 1).astype('float32')


> HERE WE CONVERT 48x48 Image To 96x96 Image

In [ ]:
import cv2

In [ ]:
def resize(x):
  
  img = x.reshape(48, 48)
  return cv2.resize(img, dsize=(96, 96), interpolation = cv2.INTER_CUBIC)

In [ ]:
facialexpression_df[' pixels'] = facialexpression_df[' pixels'].apply(lambda x: string2array(x)) # Converted To Array
facialexpression_df[' pixels'] = facialexpression_df[' pixels'].apply(lambda x: resize(x)) # Resize To 96x96

In [ ]:
facialexpression_df.head()

> Check For Null Data

In [ ]:
facialexpression_df.isnull().sum()

> Just Visualize One Image

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(facialexpression_df[' pixels'][6], cmap = 'gray')

Count How Many Emotions 

In [ ]:
facialexpression_df.emotion.value_counts().index

In [ ]:
facialexpression_df.emotion.value_counts()

In [ ]:
import seaborn as sns

In [ ]:
plt.figure(figsize = (10,10))
sns.barplot(x = facialexpression_df.emotion.value_counts().index, y = facialexpression_df.emotion.value_counts())

In [ ]:
import tensorflow as tf

X = facialexpression_df[' pixels']
y = tf.keras.utils.to_categorical(facialexpression_df['emotion'])# Convert Into Categorical Because OF CNN Output

> X's Value In Pixels And Image

In [ ]:
X[0]

In [ ]:
plt.imshow(X[0], cmap = 'gray')

> y's Value

In [ ]:
y

In [ ]:
X = np.stack(X, axis = 0)
X = X.reshape(35887, 96, 96,1)# 35887 Because Of 35887 No. Of Images

print(X.shape, y.shape)

In [ ]:
#X = np.repeat(X[..., np.newaxis], 3, -1)# For Gray-Scale To RGB Convert(1 Channel To 3 Channel Convert)

> Split X-y In To Train-Test And Validation Data

In [ ]:

from sklearn.model_selection import train_test_split

X_train, X_Test, y_train, y_Test = train_test_split(X, y, test_size = 0.1, shuffle = True)
X_val, X_Test, y_val, y_Test = train_test_split(X_Test, y_Test, test_size = 0.5, shuffle = True)

> Normalize

In [ ]:
X_train = X_train-127.5/127.5
X_val   = X_val-127.5 /127.5
X_Test  = X_Test-127.5/127.5

In [ ]:
print(X_train[0],X_val[0],X_Test[0])

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler
from IPython.display import display
from tensorflow.python.keras import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, optimizers
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import *
from tensorflow.keras import backend as K
from keras import optimizers
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

> Create Training Data-Genrator

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range = 15,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    shear_range = 0.1,
    zoom_range = 0.1,
    horizontal_flip = True,
    fill_mode = "nearest")

> We Want To Use Xception Model For Transfer Lerning In This Compition But The Data Given Is Only in 1 Channel We Converted It Into 3 Channel And Start Trainng But The Acc. Is Low And The Memory Limit Is Exided So We Desided To Build The Model

In [ ]:
'''

xception_model = Sequential()

pretrained_model= tf.keras.applications.Xception(include_top=False,
                   input_shape=(96,96,3),#Usally Is 299x299 But We Give It 96x96
                   pooling='avg',
                   weights='imagenet')
for layer in pretrained_model.layers:
        layer.trainable=False

xception_model.add(pretrained_model)
xception_model.add(Flatten())
xception_model.add(BatchNormalization())
xception_model.add(Dense(7, activation='softmax'))# For 7 Classes
xception_model.compile(loss'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

'''

> We Use Residual Blocks Here Like Resnet 

In [ ]:
def res_block(X, filter):

  # Convolutional_block
  X_copy = X

  f1 , f2, f3 = filter

  # Main Path
  X = Conv2D(f1, (1,1),strides = (1,1), kernel_initializer= glorot_uniform(seed = 0))(X)
  X = MaxPool2D((2,2))(X)
  X = BatchNormalization(axis =3,)(X)
  X = Activation('relu')(X) 

  X = Conv2D(f2, kernel_size = (3,3), strides =(1,1), padding = 'same', kernel_initializer= glorot_uniform(seed = 0))(X)
  X = BatchNormalization(axis =3)(X)
  X = Activation('relu')(X) 

  X = Conv2D(f3, kernel_size = (1,1), strides =(1,1),kernel_initializer= glorot_uniform(seed = 0))(X)
  X = BatchNormalization(axis =3)(X)


  # Short path
  X_copy = Conv2D(f3, kernel_size = (1,1), strides =(1,1), kernel_initializer= glorot_uniform(seed = 0))(X_copy)
  X_copy = MaxPool2D((2,2))(X_copy)
  X_copy = BatchNormalization(axis =3)(X_copy)

  # ADD
  X = Add()([X,X_copy])
  X = Activation('relu')(X)

  # Identity Block 1
  X_copy = X


  # Main Path
  X = Conv2D(f1, (1,1),strides = (1,1), kernel_initializer= glorot_uniform(seed = 0))(X)
  X = BatchNormalization(axis =3)(X)
  X = Activation('relu')(X) 

  X = Conv2D(f2, kernel_size = (3,3), strides =(1,1), padding = 'same', kernel_initializer= glorot_uniform(seed = 0))(X)
  X = BatchNormalization(axis =3)(X)
  X = Activation('relu')(X) 

  X = Conv2D(f3, kernel_size = (1,1), strides =(1,1), kernel_initializer= glorot_uniform(seed = 0))(X)
  X = BatchNormalization(axis =3)(X)

  # ADD
  X = Add()([X,X_copy])
  X = Activation('relu')(X)

  # Identity Block 2
  X_copy = X


  # Main Path
  X = Conv2D(f1, (1,1),strides = (1,1), kernel_initializer= glorot_uniform(seed = 0))(X)
  X = BatchNormalization(axis =3)(X)
  X = Activation('relu')(X) 

  X = Conv2D(f2, kernel_size = (3,3), strides =(1,1), padding = 'same', kernel_initializer= glorot_uniform(seed = 0))(X)
  X = BatchNormalization(axis =3)(X)
  X = Activation('relu')(X) 

  X = Conv2D(f3, kernel_size = (1,1), strides =(1,1), kernel_initializer= glorot_uniform(seed = 0))(X)
  X = BatchNormalization(axis =3)(X)

  # ADD
  X = Add()([X,X_copy])
  X = Activation('relu')(X)

  return X

In [ ]:
input_shape = (96, 96, 1)

X_input = Input(input_shape)

X = ZeroPadding2D((3, 3))(X_input)

X = Conv2D(64, (7, 7), strides= (2, 2), kernel_initializer= glorot_uniform(seed = 0))(X)
X = BatchNormalization(axis =3)(X)
X = Activation('relu')(X)
X = MaxPooling2D((3, 3), strides= (2, 2))(X)

X = res_block(X, filter= [64, 64, 256])

X = res_block(X, filter= [128, 128, 512])

X = AveragePooling2D((4, 4))(X)

X = Flatten()(X)
X = Dense(7, activation = 'softmax',kernel_initializer= glorot_uniform(seed=0))(X)

model = Model( inputs= X_input, outputs = X)

model.summary()

In [ ]:
model.compile(optimizer = "Adam", loss = "categorical_crossentropy", metrics = ["accuracy"])

In [ ]:
save = model.fit(train_datagen.flow(X_train, y_train, batch_size=100),
    validation_data=(X_val, y_val), steps_per_epoch=len(X_train) // 100,
    epochs= 20)

In [ ]:
score = model.evaluate(X_Test, y_Test)
print('Test Accuracy: {}'.format(score[1]))

> Now This Is All For icml_face Data Now We Will Acctually Train For Train Data And Pridict For Test Data

In [ ]:
train_data['pixels'] = train_data['pixels'].apply(lambda x: string2array(x)) # Converted To Array
train_data['pixels'] = train_data['pixels'].apply(lambda x: resize(x)) # Resize To 96x96

In [ ]:
train_data.head()

In [ ]:
train_data.isnull().sum()#check For Null

In [ ]:
plt.figure(figsize = (10,10))
sns.barplot(x = train_data.emotion.value_counts().index, y = train_data.emotion.value_counts())

In [ ]:
X = train_data['pixels']
y = tf.keras.utils.to_categorical(train_data['emotion'])# Convert Into Categorical Because OF CNN Output

In [ ]:
X.shape

In [ ]:
X = np.stack(X, axis = 0)
X = X.reshape(28709, 96, 96,1)# 28709 Because Of 28709 No. Of Images
print(X.shape, y.shape)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.1, shuffle = True)

In [ ]:
X_train = X_train-127.5/127.5
X_val   = X_val-127.5 /127.5

In [ ]:
save = model.fit(train_datagen.flow(X_train, y_train, batch_size=100),
    validation_data=(X_val, y_val), steps_per_epoch=len(X_train) // 100,
    epochs= 20)

In [ ]:
model.save_weights("weights.hdf5")

In [ ]:
test_data['pixels'] = test_data['pixels'].apply(lambda x: string2array(x)) # Converted To Array
test_data['pixels'] = test_data['pixels'].apply(lambda x: resize(x)) # Resize To 96x96

In [ ]:
X=test_data['pixels']

In [ ]:
X = np.stack(X, axis = 0)
X = X.reshape(7178, 96, 96,1)# 28709 Because Of 28709 No. Of Images
print(test_data.shape)

In [ ]:
y_predicted=model.predict(X)

In [ ]:
y_predicted_labels = [np.argmax(i) for i in y_predicted]
print(y_predicted_labels)

In [ ]:
submissions=pd.DataFrame({"": y_predicted_labels})
submissions.to_csv("submission.csv", index=False, header=True)

> Graphs And Etc At Last.

In [ ]:
accuracy = save.history['accuracy']
val_accuracy = save.history['val_accuracy']
loss = save.history['loss']
val_loss = save.history['val_loss']

> For Training & Validation

In [ ]:
epochs = range(len(accuracy))
plt.plot(epochs, accuracy, 'bo', label='Training Accuracy')
plt.plot(epochs, val_accuracy, 'b', label='Validation Accuracy')
plt.title('Training & Validation Accuracy')
plt.legend()

> For Loss

In [ ]:
plt.plot(epochs, loss, 'ro', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training & Validation loss')
plt.legend()

# THE END